# _prediction()_ function and the PredictionTable Class

The prediction module within SORA was created to predict stellar occultation, organize the prediction and plot the occultation maps. The documentation <a href=https://sora.readthedocs.io/>here</a> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the prediction module. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

**All functions have their Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

## 0. Index

1. [Predicting stellar occultation](#section_1)
2. [Setting and/or modifying parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_4)
5. [MPC observatories](#section_5)

<a id="section_1"></a>

## 1. Predicting stellar occultation

To predict a stellar occultation, the function `prediction()` must be used. It will need a `Body` Object and the time interval for the search. The function will generate the ephemeris within the given interval (with a default step of 60 seconds between positions, that may be changed by the user) and download the star coordinates from VizieR. The function is able to download stars from Gaia-DR2 and Gaia-EDR3. The user must select the one desired by the keys _"gaiadr2"_ and _"gaiaedr3"_ (default in this version). For every star, it checks the closest ephemeris. An occultation is identified if the distance is smaller than the radius of the Earth plus the radius of the object (given in `Body`) plus the error of the ephemeris (multiplied by a sigma factor given by the user). In the next step, the coordinate of the star is propagated to date using proper motion and parallax. It results in the occultation parameters, such as the Closest Approach distance, the Position Angle, Velocity, etc.

The function returns a `PredictionTable` with all the information.

In [1]:
from sora.prediction import prediction

SORA version: 0.2


In [2]:
prediction?

Signature:
prediction(
    time_beg,
    time_end,
    body=None,
    ephem=None,
    mag_lim=None,
    catalogue='gaiaedr3',
    step=60,
    divs=1,
    sigma=1,
    radius=None,
    verbose=True,
    reference_center='geocenter',
)
Docstring:
Predicts stellar occultations.

Parameters
----------
time_beg : `str`, `astropy.time.Time`, required
    Initial time for prediction.

time_end : `str`, `astropy.time.Time`, required
    Final time for prediction.

body : `sora.Body`, `str`, default=None
    Object that will occult the stars. It must be a Body object or its name
    to search in the Small Body Database.

ephem : `sora.Ephem`, default=None
    object ephemeris. It must be an Ephemeris object.
    If using a EphemHorizons object, please use 'divs' to make division
    at most a month, or a timeout error may be raised by the Horizon query.

mag_lim : `int`, `float`, default=None
    Faintest Gmag allowed in the search.

catalogue : `str`, default='gaiaedr3'
    The catalogue to d

**Importing and defining the** `Body` **to be used.**

As showed in the docstring of the `prediction()` function, the user can give a `Body` object, an `Ephem` object, or both (given their restrictions). Please refer to the `Body` tutorial for further explanation.

In [3]:
from sora.body import Body
from sora.ephem import EphemKernel

chariklo = Body(name='Chariklo')
ephem = EphemKernel(spkid='2010199', kernels=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'], error_ra=0.01, error_dec=0.01)
# here I'm using Body and Ephem so I'm able to give the errors in RA and DEC.
chariklo.ephem = ephem

Obtaining data for Chariklo from SBDB


In [4]:
pred = prediction(body=chariklo, time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', mag_lim=18, divs=3)

Ephemeris was split in 3 parts for better search of stars

Searching occultations in part 1/3
Generating Ephemeris between 2017-06-20 00:00:00.000 and 2017-06-23 15:59:00.000 ...
    15 Gaia-EDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 2/3
Generating Ephemeris between 2017-06-23 16:00:00.000 and 2017-06-27 07:59:00.000 ...
    12 Gaia-EDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 3/3
Generating Ephemeris between 2017-06-27 08:00:00.000 and 2017-06-30 23:59:00.000 ...
    18 Gaia-EDR3 stars downloaded
Identifying occultations ...

8 occultations found.


**Important**

The default prediction is made considering the geocenter as center of reference. However, the prediction can also be made for a specific observer. In this case, it is important to note that the search considers only the size of the object and ephemeris error. If the object is very small, consider using a time `step` for initial prediction smaller, or using a different `radius` to avoid losing a promising event. Notice the occultation parameters (closest appoach time and distance, ...) are referred to the observer.

In [5]:
from sora import Observer
outeniqua = Observer(name='Outeniqua', lon='+16 49 17.710', lat='-21 17 58.170', height=1416,
                     ephem=['input/bsp/Chariklo.bsp', 'input/bsp/de438_small.bsp'])

obs_pred = prediction(body=chariklo, time_beg='2017-06-20 00:00:00', time_end='2017-07-01 00:00:00', step=10,
                      mag_lim=18, divs=3, reference_center=outeniqua)

Ephemeris was split in 3 parts for better search of stars

Searching occultations in part 1/3
Generating Ephemeris between 2017-06-20 00:00:00.000 and 2017-06-23 15:59:50.000 ...
    10 Gaia-EDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 2/3
Generating Ephemeris between 2017-06-23 16:00:00.000 and 2017-06-27 07:59:50.000 ...
    7 Gaia-EDR3 stars downloaded
Identifying occultations ...

Searching occultations in part 3/3
Generating Ephemeris between 2017-06-27 08:00:00.000 and 2017-06-30 23:59:50.000 ...
    No star found. The region is too small or VizieR is out.

1 occultations found.


In [6]:
obs_pred[0]

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,G*,long,loct,M-G-T,S-G-T,Gaia-EDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,int64
2017-06-22 21:21:25.600,18 55 15.65251 -31 31 21.67062,18 55 15.65251 -31 31 21.67688,0.006,179.52,22.36,14.659,14.224,14.345,52,00:50,149,166,6760223758801661440


In [7]:
pred['2017-06-22 21']

Epoch,ICRS Star Coord at Epoch,Geocentric Object Position,C/A,P/A,Vel,Dist,G,G*,long,loct,M-G-T,S-G-T,Gaia-EDR3 Source ID
,,,arcsec,deg,km / s,AU,mag,mag,deg,hh:mm,deg,deg,
object,object,object,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,int64
2017-06-22 21:18:48.200,18 55 15.65251 -31 31 21.67062,18 55 15.65249 -31 31 21.62190,0.049,359.72,22.00,14.659,14.224,14.327,53,00:50,149,166,6760223758801661440


<a id="section_2"></a>

## 2. The PredictionTable

The resulting variable of the prediction function is a `PredictionTable` Object. The `PredictionTable` Class was created to have all the information about the occultations predicted, plot occultation maps and save the table in other formats.

**The** `PredictionTable` **is a class that inherits all methods from** `Astropy.table.Table` **. All the functions present in** `Astropy.table.Table` **should work for** `PredictionTable`.

More information about it at https://docs.astropy.org/en/stable/table/

**if the user asks to print the prediction, a summary of the table is shown.**

In [8]:
print(pred)

         Epoch             ICRS Star Coord at Epoch    ... Gaia-EDR3 Source ID
                                                       ...                    
----------------------- ------------------------------ ... -------------------
2017-06-21 09:57:43.440 18 55 36.17454 -31 31 19.03261 ... 6760228702284187264
2017-06-22 02:58:37.480 18 55 26.31652 -31 31 20.38440 ... 6760228839723992320
2017-06-22 21:18:48.200 18 55 15.65251 -31 31 21.67062 ... 6760223758801661440
2017-06-23 21:34:37.660 18 55 01.48119 -31 31 22.44257 ... 6760223513963694208
2017-06-24 10:13:39.980 18 54 54.06755 -31 31 22.36732 ... 6760226503261782656
2017-06-25 10:55:09.320 18 54 39.55296 -31 31 21.69987 ... 6760225163236852864
2017-06-26 08:14:35.540 18 54 26.97503 -31 31 20.51255 ... 6760226060885482624
2017-06-26 20:46:07.140 18 54 19.56971 -31 31 20.43382 ... 6760225712991422208


**to print the complete table, we must use:**

In [9]:
pred.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T Gaia-EDR3 Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg                     
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- -------------------
2017-06-21 09:57:43.440 18 55 36.17454 -31 31 19.03261 18 55 36.17500 -31 31 19.60516  0.573 179.41  21.84  14.663 15.254 15.350  225 00:56   128   165 6760228702284187264
2017-06-22 02:58:37.480 18 55 26.31652 -31 31 20.38440 18 55 26.31674 -31 31 20.74213  0.358 179.55  21.92  14.661 17.929 18.029  329 00:53   138   166 6760228839723992320
2017-06-22 21:18:48.200 18 55 15.65251 -31 31 21.67062 18 55 15.65249 -31 31 21.62190  0.049 359.72  22.00  14.659 14.224 14.327   53 00:50 

*M-G-T stands for Moon-Geocenter-Target, the on-sky angle between the star and the moon*

*S-G-T stands for Sun-Geocenter-Target, the on-sky angle between the star and the sun*

**The** `PredictionTable` **can be exported to PRAIA format file**

In this case, some information, such as the Gaia-DR2 Source ID, S-G-T and M-G-T, are lost.

In [10]:
pred.to_praia?

Signature: pred.to_praia(filename)
Docstring:
Writes PredictionTable to PRAIA format.

Parameters
----------
filename : `str`
    Name of the file to save table.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/prediction/table.py
Type:      method


In [11]:
pred.to_praia('output/Chariklo_occs.table')
# Check your folder named output

**The** `PredictionTable` **can also generate the files in the Occult Watcher feed format**

These are two files, named 'LOG.dat' and 'tableOccult_update.txt'. The occult watcher designation for the object must be informed. For instance, for the satellite Himalia, the designation is "P5M06". If `mode='append'` the prediction will be appended to an existing file. If `mode='restart'`, the file will be overwritted.

In [12]:
pred.to_ow?

Signature: pred.to_ow(ow_des, mode='append')
Docstring:
Writes PredictionTable to OccultWatcher feeder update file format.
Tables will be saved in two files: "tableOccult_update.txt" and "LOG.dat"

Parameters
----------
ow_des : `str`
    Occult Watcher designation for the object.

mode : `str`, default='append'
    Use ``'append'`` to append table to already existing file and
    ``'restart'`` to overwrite existing file.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/prediction/table.py
Type:      method


In [13]:
pred.to_ow('1997CU26')

**Using the functions from** `Astropy.table.Table` **, the data can be exported to many other formats using the funtion 'write'**

The formats allowed can be seen in https://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers. This includes latex and csv.

In [14]:
pred.write('output/prediction.tex')
# Check your folder named output

<a id="section_3"></a>

## 3. Using PredictionTable

The `PredictionTable` was not created to be instantiated directly by the user. It was designed to be an insteresting output object from the prediction function. However, a `PredictionTable` can be instantiated from a PRAIA occultation table. For this, a `PredictionTable` method must be called directly as shown below.

In [15]:
from sora.prediction import PredictionTable

In [16]:
PredictionTable.from_praia?

Signature: PredictionTable.from_praia(filename, name, **kwargs)
Docstring:
Creates a PredictionTable Table reading from a PRAIA table.

Parameters
----------
filename : `str`
    Path to the PRAIA table file.

name : `str`
    Name of the Object of the prediction.

radius : `int`, `float`, optional
    Object radius, in km.
    If not given it's searched in online database.
    When not found online, the default is set to zero.

Returns
-------
 : `sora.prediction.PredictionTable`
    A PredictionTable object.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/prediction/table.py
Type:      method


In [17]:
pred_1 = PredictionTable.from_praia(filename='output/Chariklo_occs.table', name='Chariklo')

Obtaining data for chariklo from SBDB


In [18]:
pred_1.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T  Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg            
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- ----------
2017-06-21 09:57:43.400 18 55 36.17450 -31 31 19.03260 18 55 36.17500 -31 31 19.60520  0.573 179.41  21.84  14.660 15.204 15.300  225 00:56   128   165           
2017-06-22 02:58:37.400 18 55 26.31650 -31 31 20.38440 18 55 26.31670 -31 31 20.74210  0.358 179.55  21.92  14.660 17.900 18.000  329 00:53   138   166           
2017-06-22 21:18:48.200 18 55 15.65250 -31 31 21.67060 18 55 15.65250 -31 31 21.62190  0.049 359.72  22.00  14.660 14.197 14.300   53 00:50   149   166           
2017-06-23 21:34:37.60

To remove some occultations the User can use two functions. The first is the _PredictionTable.remove_occ()_.

In [19]:
pred_1.remove_occ?

Signature: pred_1.remove_occ(date)
Docstring:
Removes stellar occultations from table.

Parameters
----------
date : `str`, `list`
    Date or list of dates of the occultation to be removed.
    The dates mut be as shown in the 'Epoch' column. If the date is not
    complete, the function will select all occultations that matches the
    given string. For instance, ``date='2020-06'`` will remove all
    occultations from the month of June 2020.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/prediction/table.py
Type:      method


In [20]:
pred_1.remove_occ('2017-06-21 09:57')

In [21]:
pred_1.pprint_all()

         Epoch             ICRS Star Coord at Epoch      Geocentric Object Position    C/A    P/A    Vel     Dist    G      G*   long  loct M-G-T S-G-T  Source ID
                                                                                      arcsec  deg   km / s    AU    mag    mag   deg  hh:mm  deg   deg            
----------------------- ------------------------------ ------------------------------ ------ ------ ------ ------- ------ ------ ---- ----- ----- ----- ----------
2017-06-22 02:58:37.400 18 55 26.31650 -31 31 20.38440 18 55 26.31670 -31 31 20.74210  0.358 179.55  21.92  14.660 17.900 18.000  329 00:53   138   166           
2017-06-22 21:18:48.200 18 55 15.65250 -31 31 21.67060 18 55 15.65250 -31 31 21.62190  0.049 359.72  22.00  14.660 14.197 14.300   53 00:50   149   166           
2017-06-23 21:34:37.600 18 55 01.48120 -31 31 22.44260 18 55 01.48120 -31 31 22.22700  0.216 359.91  22.10  14.660 17.792 17.900   48 00:46   162   167           
2017-06-24 10:13:39.90

The second function is the `PredictionTable.keep_from_selected_images()` this function will check the names in the saved images and eliminate the occultations without the map. This functions allows that after the user deleted the maps of the unwanted occultation this function will eliminate the respective rows in the `PredictionTable`.  

In [22]:
pred_1.keep_from_selected_images?

Signature: pred_1.keep_from_selected_images(path='.')
Docstring:
Keeps predictions which images were not deleted in given path.
This function uses the name of the images to identify predictions.
The name must be the automatic one generated by plot_occ_map().
The format of the image is not relevant.

Parameters
----------
path : `str`
    Path where images are located.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/prediction/table.py
Type:      method


<a id="section_4"></a>

## 4. Plotting occultation maps

SORA is also able to generate an occultation map. The only required parameters are the occultation parameters. The function also has many inputs to configure the plot.

The first time the function is called, cartopy will download some data referring to the features presented in the map, such as the country and state division, lakes, rivers, etc.

In [23]:
from sora.prediction import plot_occ_map

In [24]:
plot_occ_map?

Signature:
plot_occ_map(
    name,
    radius,
    coord,
    time,
    ca,
    pa,
    vel,
    dist,
    mag=0,
    longi=0,
    **kwargs,
)
Docstring:
Plots the map of the occultation.

Parameters
----------
name : `str`
    Name of the object.

radius : `int`, `float`
    Radius of the object, in km.

coord : `str`, `astropy.coordinates.SkyCoord`
    Coordinates of the star (``"hh mm ss.sss dd mm ss.sss"`` or
    ``"hh.hhhhhhhh dd.dddddddd"``).

time : `str`, `astropy.time.Time`
    Instant of Closest Approach (iso or isot format).

ca : `int`, `float`
    Closest Approach Distance, in arcsec.

pa : `int`, `float`
    Position Angle at C/A, in degrees.

vel : `int`, `float`
    Velocity of the event, in km/s.

dist : `int`, `float`
    Object distance at C/A, in AU.

mag : `int`, `float`, default=0
    Mag* = Normalized magnitude to vel=20km/s.

longi : `int`, `float`, default=0
    East longitude of sub-planet point, deg, positive towards East.

nameimg : `str`
    Change the name

**To make it easier to plot the predictions, the `PredictionTable.plot_occ_map()` function can be called which automatically fills the required parameters.**

To plot the map for all the predictions


In [25]:
pred.plot_occ_map(path='output/')

10199 Chariklo_2017-06-21T09:57:43.440.png generated
10199 Chariklo_2017-06-22T02:58:37.480.png generated
10199 Chariklo_2017-06-22T21:18:48.200.png generated
10199 Chariklo_2017-06-23T21:34:37.660.png generated
10199 Chariklo_2017-06-24T10:13:39.980.png generated
10199 Chariklo_2017-06-25T10:55:09.320.png generated
10199 Chariklo_2017-06-26T08:14:35.540.png generated
10199 Chariklo_2017-06-26T20:46:07.140.png generated


To plot the map for only one prediction, just give a item to the `PredictionTable` object.

In [26]:
pred[0].plot_occ_map(path='output/')

10199 Chariklo_2017-06-21T09:57:43.440.png generated


**The** `PredictionTable` **can also be plotted by giving the date of the occultation. All the occultations that matches the date will be plotted.**

The date can be as constrained as the user wants, and must match the text that appears in the _'Epoch'_ column.

In [27]:
pred['2017-06-26'].plot_occ_map()

10199 Chariklo_2017-06-26T08:14:35.540.png generated
10199 Chariklo_2017-06-26T20:46:07.140.png generated


<a id="section_5"></a>

## 5. Occultation parameters

Finally, a function is implemented in the prediction module which calculates the occultation parameters. For this function, it must be passed a `Star` Object, an `Ephem` Object (it can be any of the `Ephem` classes) and a time. The time does not need to be precise, but it must be close within 60 minutes from the Closest Approach time.

In [28]:
from sora.prediction import occ_params

In [29]:
occ_params?

Signature:
occ_params(
    star,
    ephem,
    time,
    n_recursions=5,
    max_tdiff=None,
    reference_center='geocenter',
)
Docstring:
Calculates the parameters of the occultation, as instant, CA, PA.

Parameters
----------
star : `sora.Star`
    The coordinate of the star in the same reference frame as the ephemeris.
    It must be a Star object.

ephem : `sora.Ephem*`
    Object ephemeris. It must be an Ephemeris object.

time : `astropy.time.Time`
    Time close to occultation epoch to calculate occultation parameters.

n_recursions : `int`, default=5
    The number of attempts to try obtain prediction parameters in case the
    event is outside the previous range of time.

max_tdiff : `int`, default=None
    Maximum difference from given time it will attempt to identify the
    occultation, in minutes. If given, 'n_recursions' is ignored.

reference_center : `str`, `sora.Observer`, `sora.Spacecraft`
        A SORA observer object or a string 'geocenter'.
        The occultati

In [30]:
from sora.star import Star

s = Star(code='6760225712991422208', verbose=False)

In [31]:
tca, ca, pa, vel, dist = occ_params(star=s, ephem=chariklo.ephem, time='2017-06-26 20:40')

print('Time of the CA:   {} UTC'.format(tca))
print('Closest Approach: {:.3f}'.format(ca))
print('Position Angle:   {:.3f}'.format(pa))
print('Shadow Velocity:  {:.3f}'.format(vel))
print('Object Distance:  {:.5f}'.format(dist))

Time of the CA:   2017-06-26 20:46:07.140 UTC
Closest Approach: 0.199 arcsec
Position Angle:   0.511 deg
Shadow Velocity:  -22.333 km / s
Object Distance:  14.65229 AU


**This Jupyter-Notebook was designed as a tutorial for how to work with the** `prediction()` **function and** `PredictionTable` **Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

**The End**